# 2025 in Frames

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/editor/creative/2025_in_Frames.ipynb)

### A Personalized Video Year in Review

Turn your video analytics into a shareable 33-second recap video.

This notebook creates a cinematic journey through your 2025 video stats:
- Total minutes uploaded with growth metrics
- Search activity visualization
- Clips generated via automation
- Scenes and frames analyzed
- All set to music with dynamic video grids and text overlays

All powered by **VideoDB's Editor SDK** - turning data into a visual story through code.

---

## 🚀 Setup

First, let's install the VideoDB SDK that powers all our video editing magic.

In [1]:
%pip -q install videodb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### Import Required Libraries

We'll import all the necessary modules from VideoDB for video handling, editing, and playback.

In [2]:
import json
import random
import videodb
import os

from videodb import MediaType, play_stream
from getpass import getpass
from videodb import play_stream

### Connect to VideoDB

Enter your API key to authenticate and access your video collection.

In [3]:
# Connect to VideoDB
api_key = getpass("🔑 Enter your VideoDB API Key: ")
os.environ["VIDEO_DB_API_KEY"] = api_key

conn = videodb.connect()
coll = conn.get_collection()

print("✅ Connected to VideoDB!")

🔑 Enter your VideoDB API Key: ··········
✅ Connected to VideoDB!


---

## 📊 Step 1: Calculate Your Metrics

Time to crunch the numbers! We'll fetch all your uploaded videos and calculate key metrics like total minutes indexed, clips generated, and scenes analyzed.

These metrics will be transformed into the visual story throughout the video.

In [ ]:
videos = coll.get_videos()

TOTAL_SECONDS = 0
for v in videos:
  TOTAL_SECONDS += v.length

# Core metrics
TOTAL_MINUTES = round(TOTAL_SECONDS / 60, 1)
# Below matrix can be tracked from usage history of user.
TOTAL_SEARCHES = 2573
CLIPS_GENERATED = 574
SCENES_ANALYZED = 248
FRAMES_ANALYZED = 3200

# Previous year comparison (for growth %)
PREV_YEAR_MINUTES = 6400

# Upload time conversions
TOTAL_HOURS = round(TOTAL_MINUTES / 60, 1)
TOTAL_DAYS = round(TOTAL_HOURS / 24, 1)

# Growth percentages
MINUTES_GROWTH = round(((TOTAL_MINUTES - PREV_YEAR_MINUTES) / PREV_YEAR_MINUTES) * 100)

---

## 🎵 Step 2: Upload Background Music

Every great video needs a soundtrack! Upload your background music track that will play throughout the entire recap video.

In [ ]:
# upload audio

audio = coll.upload("https://youtu.be/vIEi8AoP_Fw?si=AUVUt6Komqtt7rWI", media_type="audio")

---

## 🎬 Step 3: Initialize Timeline and Tracks

Now we set up the foundation of our video:
- Create the main **Timeline** with resolution and background color
- Set up separate **Tracks** for text overlays and video clips
- Prepare to layer our content like a professional video editor

In [ ]:
from videodb.editor import (
    Timeline,
    Track,
    Clip,
    VideoAsset,
    AudioAsset,
    TextAsset,
    Offset,
    Transition,
    Font,
    Filter,
)

videos = videos[20:40]

timeline = Timeline(conn)
timeline.background = "#E85E00"
track = Track()
video_track = Track()

---

## 🎼 Step 4: Add Background Music Track

Add the background music that will play throughout the video, creating the emotional foundation for our visual story.

In [ ]:
# Add backgound music

b_music = Clip(
    asset=AudioAsset(id=audio.id, start=0, volume=1),
    duration=28,
)
track.add_clip(4, b_music)

---

## ✨ Step 5: Create the Intro Section

**Duration:** 0-5 seconds

The opening moment that hooks viewers! We create:
- A bold intro text: *"2025 was a blur. Let's index it."*
- 60 video clips arranged in a cascading grid with varying scales and positions
- Creates a dynamic, eye-catching mosaic effect that builds anticipation

In [ ]:
# intro text

intro_text = Clip(
    asset=TextAsset(
        text="2025 was a blur. Let's index it.",
        font=Font(size=60, family="Roboto Bold"),
    ),
    duration=5,
    transition=Transition(in_="fade"),
)
track.add_clip(1, intro_text)

# Create 20 video clips with 1 second duration each and different scales
# Scales range from 0.5 to 0.95
for i in range(60):
    scale_value = 0.5 + (i * 0.025)  # Scale: 0.5, 0.525, 0.55, ... 0.975
    start_time = random.randint(0, 4)
    volume = 0.5
    if start_time == 3:
        volume = 0.3
    elif start_time == 4:
        volume = 0.3

    # Pick a random video for each clip
    video = random.choice(videos)

    video_clip = Clip(
        asset=VideoAsset(
            id=video.id,
            start=random.randint(
                10, int(video.length - 10)
            ),  # Start from different positions in the source video
            volume=volume,
        ),
        duration=1,  # 1 second each
        scale=scale_value,
        fit=None,
        offset=Offset(x=random.uniform(-0.5, 0.5), y=random.uniform(-0.5, 0.5)),
    )

    # Add each clip at sequential 1-second intervals
    video_track.add_clip(start=start_time, clip=video_clip)

---

## 📈 Step 6: Section 1 - Upload Metrics

**Duration:** 6-13 seconds

Showcase your upload statistics with a dramatic **zoom-out effect**:
- Display total minutes, hours, and days of content uploaded
- Show year-over-year growth percentage
- 20 video clips that start large and scale down, creating a cinematic reveal
- Greyscale filter adds a sophisticated, professional look

In [ ]:
# section 1st

section1_text = Clip(
    asset=TextAsset(
        text=f"You uploaded {TOTAL_MINUTES:,} minutes\nof video content.\n\nThat's {TOTAL_HOURS} hours.\nOr {TOTAL_DAYS} full days of footage.\n\n↑ {MINUTES_GROWTH}% from 2024",
        font=Font(size=48, family="default"),
    ),
    duration=7,
)
track.add_clip(6, section1_text)

# Zoom out effect for section 1 using video assets
# Videos start large and scale down (zoom out effect)
for i in range(20):
    scale_value = 0.5 + (i * 0.0263)  # Scale: 0.5, 0.526, ... 1.0 (zoom out effect)
    start_time = 6 + (i * 0.175)  # Spread across section 1 duration (7 seconds)

    # Pick a random video for each clip
    video = random.choice(videos)

    # Safe video start time
    video_start = random.randint(10, max(11, int(video.length - 10)))

    video_clip = Clip(
        asset=VideoAsset(
            id=video.id,
            start=video_start,
            volume=0.15,
        ),
        duration=1,
        scale=scale_value,
        fit="crop",
        filter=Filter.greyscale,
    )

    video_track.add_clip(start=start_time, clip=video_clip)

---

## 🔍 Step 7: Section 2 - Search Analytics

**Duration:** 13-19 seconds

Visualize your search activity with a **horizontal scanning effect**:
- Display the total number of questions asked
- 3 rows of video clips moving horizontally at different speeds
- Creates a dynamic "data stream" visual that represents video intelligence in action

In [ ]:
# secction 2nd

section2_text = Clip(
    asset=TextAsset(
        text=f"{TOTAL_SEARCHES:,} questions asked.\n\nYour videos aren't just stored—\nthey're understood.",
        font=Font(size=58, family="default"),
    ),
    duration=6,
)
track.add_clip(13, section2_text)

rows = [
    {"y": -0.5, "num_clips": 20, "duration": 0.25, "scale": 0.35},  # Top - fastest
    {"y": 0, "num_clips": 15, "duration": 0.33, "scale": 0.45},  # Middle - medium
    {"y": 0.5, "num_clips": 12, "duration": 0.42, "scale": 0.35},  # Bottom - slowest
]

clip_idx = 0
for row in rows:
    for i in range(row["num_clips"]):
        x_pos = -0.9 + (i * 0.1)  # Move left to right
        video = videos[clip_idx % len(videos)]

        # Safe video start time - make sure it doesn't exceed video length
        video_start = 30 + (clip_idx % 10)  # Cycle through 30-40 seconds
        if video_start > video.length - 10:
            video_start = 15  # Fallback to safe position

        video_clip = Clip(
            asset=VideoAsset(
                id=video.id,
                start=video_start,
                volume=0.15,
            ),
            duration=row["duration"],
            scale=row["scale"],
            fit="crop",
            offset=Offset(x=x_pos, y=row["y"]),
        )
        video_track.add_clip(start=12 + (i * row["duration"]), clip=video_clip)
        clip_idx += 1


section3_text = Clip(
    asset=TextAsset(
        text=f"{CLIPS_GENERATED:,} clips\ngenerated via code.\n\nAutomation at its finest.",
        font=Font(size=62, family="default"),
    ),
    duration=6,
)
track.add_clip(19, section3_text)

---

## 🤖 Step 8: Section 3 - Automation Stats

**Duration:** 19-25 seconds

Highlight the power of automation with a **cycling background effect**:
- Showcase total clips generated via code
- Large, cycling background clips create a sense of scale and automation
- Emphasizes the "code-first" approach to video creation

In [ ]:
# section 3rd

section3_text = Clip(
    asset=TextAsset(
        text=f"{CLIPS_GENERATED:,} clips\ngenerated via code.\n\nAutomation at its finest.",
        font=Font(size=62, family="default"),
    ),
    duration=6,
)
track.add_clip(19, section3_text)

# Add continuous cycling large background clip throughout section 3
for i in range(3):
    video = videos[(i + 5) % len(videos)]
    video_start = 20 + (i * 15)
    if video_start > video.length - 10:
        video_start = 10

    bg_clip = Clip(
        asset=VideoAsset(
            id=video.id,
            start=video_start,
            volume=0.05,
        ),
        duration=2,
        scale=1.2,
        fit="crop",
        filter=Filter.greyscale,
        opacity=0.2,
        z_index=0,
    )
    video_track.add_clip(start=19 + (i * 2), clip=bg_clip)

---

## 🧠 Step 9: Section 4 - Intelligence Metrics

**Duration:** 25-31 seconds

Display video intelligence capabilities with a **scanning grid animation**:
- Show scenes and frames analyzed
- 3x3 grid of video clips appears sequentially with scanning effect
- Two columns of cascading clips create a "data processing" visual
- Alternating contrast and boost filters add visual rhythm

In [ ]:
# section 4th

section4_text = Clip(
    asset=TextAsset(
        text=f"{SCENES_ANALYZED:,} scenes analyzed\n{FRAMES_ANALYZED:,} Frames Analyzed\n\nPure video intelligence.\nPowered by you.",
        font=Font(size=52, color="#FFFFFF", family="Arial Bold"),
    ),
    duration=5,
)
track.add_clip(25, section4_text)

# Section 4 animation: Improved grid with continuous background
# Create a 3x3 grid of video clips that appear sequentially
grid_positions = [
    (-0.4, -0.4),
    (0, -0.4),
    (0.4, -0.4),  # Top row
    (-0.4, 0),
    (0, 0),
    (0.4, 0),  # Middle row
    (-0.4, 0.4),
    (0, 0.4),
    (0.4, 0.4),  # Bottom row
]

# Main grid appears with scanning effect
for idx, (x_pos, y_pos) in enumerate(grid_positions):
    video = videos[idx % len(videos)]

    # Safe video start
    video_start = 50 + (idx % 10)
    if video_start > video.length - 10:
        video_start = 20

    # Create scanning effect with transitions and filters
    video_clip = Clip(
        asset=VideoAsset(
            id=video.id,
            start=video_start,
            volume=0.1,
        ),
        duration=0.6,
        scale=0.28,
        fit="crop",
        offset=Offset(x=x_pos, y=y_pos),
        transition=Transition(in_="fade", duration=0.15),
        filter=Filter.contrast if idx % 2 == 0 else Filter.boost,
        opacity=0.9,
    )

    # Stagger the appearance for scanning effect
    video_track.add_clip(start=24.5 + (idx * 0.08), clip=video_clip)

num_columns = 2
clips_per_column = 8
for col in range(num_columns):
    x_pos = -0.3 + (col * 0.6)  # Two columns centered in viewport

    for row in range(clips_per_column):
        y_pos = -0.7 + (row * 0.23)  # Top to bottom
        clip_idx = col * clips_per_column + row

        video = videos[clip_idx % len(videos)]

        # Safe video start
        video_start = 40 + (clip_idx % 8)
        if video_start > video.length - 10:
            video_start = 25

        video_clip = Clip(
            asset=VideoAsset(
                id=video.id,
                start=video_start,
                volume=0.15,
            ),
            duration=0.3,
            scale=0.3,
            fit="crop",
            offset=Offset(x=x_pos, y=y_pos),
        )
        # Stagger between columns for cascading effect
        video_track.add_clip(start=25.7 + (row * 0.3) + (col * 0.1), clip=video_clip)

---

## 🎯 Step 10: Create the Outro

**Duration:** 31-33 seconds

End with a forward-looking call to action: *"Ready for 2026? Build it."*

A simple, powerful message that leaves viewers inspired.

In [ ]:
# outro text

outro_text = Clip(
    asset=TextAsset(
        text="Ready for 2026? Build it.", font=Font(size=60, family="Roboto Bold")
    ),
    duration=2,
    # transition=Transition(in_="fade"),
)
track.add_clip(31, outro_text)

---

## 🎥 Step 11: Compile and Generate the Final Video

Time to bring it all together! We'll:
1. Add all tracks to the timeline (video track first, then text track for proper layering)
2. Generate the final stream URL
3. Preview the complete 33-second journey through your 2025 achievements

Watch as your data transforms into a cinematic experience!

In [ ]:
timeline.add_track(video_track)
timeline.add_track(track)


stream_url = timeline.generate_stream()
print(stream_url)
play_stream(stream_url)

https://d1zudc7ewmc6ey.cloudfront.net/v1/a0f7b1cd-26b1-4c1b-b2d8-deaef0be5524.m3u8


---

## That's a Wrap!

We just turned raw analytics into a 33-second cinematic journey - all with code!

### What we built:
- **Multi-layered timeline** - Separate tracks for audio, visuals, and text
- **Procedural animation** - 100+ video clips positioned through code
- **Dynamic effects** - Zoom transitions, scanning grids, cascading animations
- **Data storytelling** - Metrics transformed into engaging visuals

### The Magic of VideoDB Editor SDK:
- No manual editing - everything defined in Python
- Precise control over positioning, timing, and transitions
- Reusable patterns for different data sets
- Generate multiple versions by changing input metrics

**Want your own recap?** Just update the metrics in Step 1 and run the cells!

---

*Made with VideoDB Editor SDK*